In [1]:
import os
import pandas as pd
import random
import json
import numpy as np
import cv2

df = pd.read_csv('../annotations.csv')

#size =  # Set size of dataset
# imgs = range(2,7640)

with open('acino_all.json','r+') as f:
    anns_data=json.load(f)
    
    for i in range(0,len(df)):# Iterate over each image

        #Add image instance
        img_name=df.iloc[i,0]
        img_id=int(img_name[3:].split('.')[0])
        img=cv2.imread('../data/'+img_name)
        height = img.shape[0]
        width = img.shape[1]
        img_inst=dict(width=width,height=height,file_name=img_name,background=1,id=img_id)
        
        #Add annotation instance
        ann_id=img_id
        kp=[]
        num_kp=0

        # Initial bbox:
        x_min=width-1
        x_max=1
        y_min=height-1
        y_max=1

        for j in range(1,len(df.columns),2): # range(1,55) for 
            if pd.isnull(df.iloc[i,j]): #NaN values
                kp.append(0)
                kp.append(0)
                kp.append(0)
                
            else:
                x=int(round(pd.to_numeric(df.iloc[i,j])))
                y=int(round(pd.to_numeric(df.iloc[i,j+1])))
                
                #Add annotations
                kp.append(x)
                kp.append(y)
                kp.append(2)
                num_kp+=1

                #Bbox
                if x<x_min:
                    x_min=x
                if x>x_max:
                    x_max=x
                if y>y_max:
                    y_max=y
                if y<y_min:
                    y_min=y

        w=max(1,(x_max-x_min)) # BBox width
        h=max(1,(y_max-y_min)) # BBox height
        a=w*h # Bbox area
        
        #Padded bounding box (5 pixels all sides):
        bbox=[x_min-10,y_min-10,w+20,h+20] # xywh format NB! y_min==top left corner
        
#         if not kp[-1]==0: # Tip of tail != NaN
#             num_kp-=1 
            
#         del kp[-3:] # Remove tail_x, tail_y, visible
        
        
        ann_inst=dict(image_id=img_id,iscrowd=0,category_id=25,num_keypoints=num_kp,keypoints=kp,bbox=bbox,id=ann_id,area=a)
        anns_data['images'].append(img_inst)
        anns_data['annotations'].append(ann_inst)
    f.seek(0)
    json.dump(anns_data,f,indent=4)
f.close()

In [2]:
%pip install echo1-coco-split

Note: you may need to restart the kernel to use updated packages.


In [3]:
!coco-split --annotations_file acino_all.json --valid_ratio 0.1 --test_ratio 0.2 --train_name acino_train.json --valid_name acino_val.json --test_name acino_test.json --has_annotations

Saved 5346 annotations from 5346 images to acino_train.json
Saved 764 annotations from 764 images to acino_val.json
Saved 1528 annotations from 1528 images to acino_test.json
